# Loading data

In [1]:
import json

with open('clinais.train.json') as f:
    data = json.load(f)

from tqdm import tqdm

finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

100%|██████████| 781/781 [00:00<00:00, 8777.89it/s]


In [2]:
from datasets import load_from_disk

dataset = load_from_disk('augmented2')

/home/joheras/.local/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


# Processing dataset

In [3]:
from transformers import AutoTokenizer

In [4]:
modelCheckpoint = "joheras/bsc-bio-ehr-es-finetuned-clinais"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
tokenized_dataset['train'] = tokenized_dataset['train'].remove_columns('__index_level_0__')

In [10]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
import evaluate

seqeval = evaluate.load("seqeval")

In [12]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [13]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on

In [14]:
training_args = TrainingArguments(
    output_dir="bsc-bio-ehr-es-finetuned-clinais-augmented2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/joheras/.local/lib/python3.10/site-packages/huggingface_hub/repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(
Cloning https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented2 into local empty directory.
The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3624
  Num Epochs = 100
  Instantaneous batch si

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.755010,0.116564,0.160507,0.135051,0.793702
2,No log,0.657677,0.209604,0.290391,0.243471,0.812330
3,No log,0.592850,0.216667,0.315734,0.256983,0.819652
4,No log,0.583682,0.253173,0.379092,0.303594,0.837404
5,0.504500,0.575772,0.330872,0.520591,0.404596,0.847891
6,0.504500,0.626594,0.350603,0.521647,0.419355,0.838450
7,0.504500,0.632885,0.350331,0.558606,0.430606,0.848626
8,0.504500,0.625097,0.376853,0.563886,0.451777,0.856541
9,0.104300,0.650895,0.399858,0.594509,0.478132,0.858152
10,0.104300,0.743136,0.375519,0.573390,0.453824,0.845319


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tags, tokens. If tags, tokens are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 32
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to bsc-bio-ehr-es-finetuned-clinais-augmented2/checkpoint-114
Configuration saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/checkpoint-114/config.json
Model weights saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/checkpoint-114/pytorch_model.bin
tokenizer config file saved in bsc-bio-ehr-es-finetu

TrainOutput(global_step=11400, training_loss=0.03091640188505775, metrics={'train_runtime': 7034.2912, 'train_samples_per_second': 51.519, 'train_steps_per_second': 1.621, 'total_flos': 9.44647381474416e+16, 'train_loss': 0.03091640188505775, 'epoch': 100.0})

In [15]:
trainer.push_to_hub()

Saving model checkpoint to bsc-bio-ehr-es-finetuned-clinais-augmented2
Configuration saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/config.json
Model weights saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/pytorch_model.bin
tokenizer config file saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/tokenizer_config.json
Special tokens file saved in bsc-bio-ehr-es-finetuned-clinais-augmented2/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/473M [00:00<?, ?B/s]

Upload file runs/Mar21_15-58-12_minion/events.out.tfevents.1679410700.minion.474056.0:  59%|#####8    | 32.0k/…

To https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented2
   7f147fc..48c342c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.4987755102040816}, {'name': 'Recall', 'type': 'recall', 'value': 0.6451953537486801}, {'name': 'F1', 'type': 'f1', 'value': 0.5626151012891344}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8592831298055179}]}
To https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented2
   48c342c..6d5b0d7  main -> main



'https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented2/commit/48c342ca070b66f16b64d10ecdbb5e73ba764e59'

In [16]:
!rm -rf bsc-bio-ehr-es-finetuned-clinais-augmented2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
